In [45]:
library(tidyverse)
library(ggrepel)

In [26]:
load_population_diversity <- function(path){
    
    city <- dirname(dirname(path))
    base <- basename(path)
    pop <- str_extract(base, '(?<=\\w_)\\d+(?=_4fold)')
    
    full_path <- paste0(inpath, '/', path)
    df <- suppressMessages(read_delim(full_path, delim = '\t')) %>% 
        mutate(city = city,
               pop = as.character(pop),
               tp_scaled = tP / nSites) %>% 
    dplyr::select(city, pop, tp_scaled, nSites)
    return(df)
}

In [27]:
inpath <- '../results/angsd/summary_stats/thetas/by_city/'
pop_div_df <- list.files(inpath, pattern = '.*_\\d+_4fold.thetas.idx.pestPG', recursive = TRUE) %>% 
    map_dfr(., load_population_diversity)

In [29]:
habitats <- suppressMessages(read_delim('../../sequencing-prep/resources/low1_sampleSheet.txt', delim = '\t')) %>% 
    mutate(pop = as.character(pop))
pop_div_df <- pop_div_df %>% 
    left_join(., habitats, by = c('city', 'pop'))

In [53]:
pop_div_df %>% 
    group_by(city, site) %>% 
    summarize(mean = mean(tp_scaled),
              se = sd(tp_scaled) / sqrt(n())) %>% 
    ungroup() %>% 
    ggplot(., aes(x = site, y = mean, color = city, label = city)) +
        geom_point(size = 1) + 
#         geom_errorbar(aes(ymax = mean + se, ymin = mean - se), width = 0.15) +
        geom_line(aes(group = city)) +
        geom_text_repel(point.size = NA) +
        theme_classic() + theme(legend.position = 'none')